In [1]:
import numpy as np
import os
import math

In [2]:
outputdir = "../output/"

In [3]:
# 定数の定義
pi = 3.14159265359
a = 1.461
root3 = 1.73205080756
c2of3 = 0.66666666666666
a1 = np.array([a, 0.0, 0.0])
a2 = np.array([a * 0.5, a * 0.5 * root3, 0.0])
A = np.array([0.0, 0.0, 0.0])
B = np.array([c2of3, c2of3, 0.0])

bondl = a / root3
bond_error = bondl / 10
judge_bondl = bondl + bond_error

In [4]:
def funcx(a, b, y):
    return a * y + b


def funcy(a, b, x):
    return a * x + b


def examin_the_size(a, b, vari):
    temp = sorted([a, b])
    return 1 if temp[0] < vari <= temp[1] else 0

In [5]:
class Atom:
    def __init__(self, coord, atom_id):
        self.coord = np.array(coord)
        self.id = atom_id

    def getx(self):
        return self.coord[0]

    def gety(self):
        return self.coord[1]

    def getz(self):
        return self.coord[2]

    def getid(self):
        return self.id

    def putx(self, x):
        self.coord[0] = x

    def puty(self, y):
        self.coord[1] = y

    def putz(self, z):
        self.coord[2] = z

    def getcoord(self):
        return self.coord

In [6]:
class Bond:
    def __init__(self, a0, a1):
        self.coord = (np.array(a0) + np.array(a1)) / 2
        self.angle = np.zeros(3)
        self.angle[0] = math.atan((a1[1] - a0[1]) / (a1[2] - a0[2]))
        self.angle[1] = math.atan((a1[0] - a0[0]) / (a1[2] - a0[2]))
        self.angle[2] = 0.0

    def getcoord(self):
        return self.coord

    def getx(self):
        return self.coord[0]

    def gety(self):
        return self.coord[1]

    def getz(self):
        return self.coord[2]

    def getanglex(self):
        return self.angle[0]

    def getangley(self):
        return self.angle[1]

    def getanglez(self):
        return self.angle[2]

In [7]:
class NanoTube:
    def __init__(self, n, m, length):
        self.n = n
        self.m = m
        self.length = length
        self.ch = n * a1 + m * a2
        ro90 = np.array([
            [0, -1 * length / np.linalg.norm(self.ch), 0],
            [1 * length / np.linalg.norm(self.ch), 0, 0],
            [0, 0, 0]
        ])
        self.lt = np.dot(ro90, self.ch)
        self.r = np.linalg.norm(self.ch) * 0.5 / pi
        self.Atoms = []
        self.Bonds = []

    def graphene(self, aid, bid):
        L1, L2 = 1000, 1000
        ac = self.ch[1] / self.ch[0]
        al = self.lt[0] / self.lt[1]
        theta = math.atan(ac)
        roma = np.array([
            [math.cos(theta), math.sin(theta), 0],
            [-math.sin(theta), math.cos(theta), 0],
            [0, 0, 1]
        ])

        for i in range(-L1, L1):
            for j in range(-L2, L2):
                # Put atom A
                b = np.array([i + A[0], j + A[1]])
                c = a1 * b[0] + a2 * b[1]

                if (
                    examin_the_size(funcy(ac, 0.0, c[0]), funcy(ac, self.lt[1] - ac * self.lt[0], c[0]), c[1]) and
                    examin_the_size(funcx(al, 0.0, c[1]), funcx(al, self.ch[0] - al * self.ch[1], c[1]), c[0])
                ):
                    self.Atoms.append(Atom(c, aid))

                # Put atom B
                b = np.array([i + B[0], j + B[1]])
                c = a1 * b[0] + a2 * b[1]

                if (
                    examin_the_size(funcy(ac, 0.0, c[0]), funcy(ac, self.lt[1] - ac * self.lt[0], c[0]), c[1]) and
                    examin_the_size(funcx(al, 0.0, c[1]), funcx(al, self.ch[1] - al * self.ch[0], c[1]), c[0])
                ):
                    self.Atoms.append(Atom(c, bid))

        for atom in self.Atoms:
            tempp = np.dot(roma, atom.getcoord())
            atom.putx(tempp[0])
            atom.puty(tempp[1])
            atom.putz(tempp[2])

    def tube(self):
        for atom in self.Atoms:
            theta = atom.getx() / self.r
            atom.putx(self.r * math.cos(theta))
            atom.putz(self.r * math.sin(theta))

    def bond(self):
        for i in range(len(self.Atoms) - 1):
            for j in range(i + 1, len(self.Atoms)):
                bond_vec = self.Atoms[i].getcoord() - self.Atoms[j].getcoord()
                if np.linalg.norm(bond_vec) <= judge_bondl:
                    self.Bonds.append(Bond(self.Atoms[i].getcoord(), self.Atoms[j].getcoord()))

    def xyz(self, filename="default.xyz"):
        with open(filename, "w") as outputxyz:
            outputxyz.write(f"{len(self.Atoms)}\n")
            outputxyz.write("id,posx,posy,posz\n")
            for atom in self.Atoms:
                outputxyz.write(f"{atom.getid()} {atom.getx()} {atom.gety()} {atom.getz()}\n")


In [8]:
n = 6
m = 5

In [9]:
id1 = "C"
id2 = "C"

In [10]:
length = 10

In [11]:
nanotube = NanoTube(n, m, length)
nanotube.graphene(id1, id2)
nanotube.tube()
nanotube.xyz(os.path.join(outputdir, f"nanotube_{n}_{m}.xyz"))